In [7]:
import pandas as pd
from disk_analyzer.utils.constants import *

# Сбор данных

In [ ]:
from disk_analyzer.stages import DataCollector

Для начала создадим объект, который соберёт из сырых данных, расположенных по путям path1, path2, path3... данные разбитые на батчи. Сырые данные - это распакованные архивы с сайта [Backblaze](https://www.backblaze.com/cloud-storage/resources/hard-drive-test-data). Собранные данные находятся в папке storage_path.

In [ ]:
paths = ['Data/Sources']
storage_path = 'Data/Data_collected'
batchsize = BATCHSIZE
dc = DataCollector(paths, storage_path, batchsize=batchsize)
dc.collect_data()

# Препроцессинг

In [ ]:
from disk_analyzer.stages import ModelPipeline
from disk_analyzer.models.DLClassifier import DLClassifier
from disk_analyzer.models import SKLearnClassifier

Теперь, данные, которые разбиты на батчи необходимо предобработать, разбить на train и test и т.д. Если во время этого процесса ноутбук крашится, значит машина не тянет такое количество данных и нужно из папки с батчами удалить часть данных. DataPreprocessor берёт все данные из папки storage_path и создаёт новую папку с данными разбитыми на train и test.

In [11]:
# Для начала создадим класс, который воплощает в себе логику всего пайплайна модели
MP = ModelPipeline()

In [12]:
import glob as glob
import os
data_paths = glob.glob(os.path.join(storage_path, '*.csv'))
MP.preprocess(data_paths)

Fit_tr TimeTransformer
Fit_tr DropDoubles


/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.time_column] = X[self.time_column].astype('datetime64[ns]')
/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'time'] = (X[self.time_column] - X.groupby('serial_number')
/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:273: SettingWithCop

Fit_tr DropDuplicates
Fit_tr FeatureFilter
Fit_tr Nan_imputer
Fit_tr CategoricalEncoder
Fit_tr StandardScaler
Fit_tr RandomSampler
Fit_tr LabelShifter
Fit_tr TimeLabeler
Applying TimeTransformer


/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.time_column] = X[self.time_column].astype('datetime64[ns]')
/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'time'] = (X[self.time_column] - X.groupby('serial_number')
/home/dvarfe/Documents/HW/Backblaze_ML_Ops/disk_analyzer/stages/data_preprocessor.py:273: SettingWithCop

Applying DropDoubles
Applying DropDuplicates
Applying FeatureFilter
Applying Nan_imputer
Applying CategoricalEncoder
Applying StandardScaler
Applying RandomSampler
Applying LabelShifter
Applying TimeLabeler
Save Preprocessed Data


'./Data/preprocessed/test'

# Обучение

Перед тем как обучить модель, необходимо передать её в пайплайн(если она не была задана заранее, например, из файла), с помощью set_model

In [13]:
dl_model = DLClassifier(FEATURES_NUM)
MP.set_model(dl_model, interface='torch')

In [14]:
#Будем обучаться на одном файле
fit_paths = ['Data/preprocessed/train/0_preprocessed.csv']
# Если fit падает с ошибкой - поменяйте значение переменной FEATURES_NUM в файле constants. Управлять числом эпох можно там же, с помощью EPOCHS.
MP.fit(fit_paths)

Epoch 1/1 - Loss: 8.2530


In [15]:
# Считать качество будем на обучающей выборке
ci, ibs = MP.score_model(fit_paths)

 90%|████████▉ | 1009/1124 [01:06<00:07, 15.21it/s]


In [16]:
print(f'CI={ci}, IBS={ibs}')

CI=0.7169051387465166, IBS=0.5501861675527848
